## Import Libraries

In [1]:
from neo4j import GraphDatabase

## Database Configuration

In [2]:
# Load config from file
def load_neo4j_config(filepath):
    config = {}
    with open(filepath, 'r') as file:
        for line in file:
            if '=' in line:
                key, value = line.strip().split('=', 1)
                config[key] = value
    return config

In [3]:
# Load credentials from the file
config = load_neo4j_config("../../Encryption/Stack Overflow Sandbox Credentials.txt")

In [4]:
# Assign values
uri = config.get("NEO4J_URI")
username = config.get("NEO4J_USERNAME")
password = config.get("NEO4J_PASSWORD")
database = config.get("NEO4J_DATABASE", "neo4j")

## Database Class

In [5]:
class CypherTester:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def initialise_graph(self):
        with self.driver.session() as session:
            session.run("MATCH (n) DETACH DELETE n") # Clear any existing data from the database.

            session.run("""
                CREATE (john: Person {name: "John"})
                CREATE (sally: Person {name: "Sally"})
                CREATE (graphs: Topic {name: "Graph"})
                CREATE (neo: Company {name: "Neo4j"})
                CREATE (sally) - [:IS_FRIENDS_WITH] -> (john)
                CREATE (sally) - [:LIKES] -> (graphs)
                CREATE (sally) - [:WORKS_FOR] -> (neo)            
            """
            )

            print("Graph Initialised")

    def run_query(self, query):
        with self.driver.session() as session:
            result = session.run(query)
            return [record.values() for record in result]

    def test_friend_of_sally(self):
        query = """
            MATCH (s:Person {name: 'Sally'})-[:IS_FRIENDS_WITH]->(friend)
            RETURN friend.name
        """

        return self.run_query(query)

In [6]:
tester = CypherTester(uri, username, password)

tester.initialise_graph()
print(f"Sally's friends: {tester.test_friend_of_sally()}")

Graph Initialised
Sally's friends: [['John']]
